## KIM Content development using the KIM Developer Platform

Everything we have done so far can be performed on any system with the KIM API, packages, and simulators installed. However, this Binder is based on the [KIM Developer Platform (KDP)](https://openkim.org/doc/evaluation/kim-developer-platform/), a Docker image for KIM development.

So far, we have been installing models using `kim-api-collections-management`, which only retains the binaries for use. Let's install a Tersoff potential for carbon using the `kimitems` utility, which is only available in the KDP:

In [ ]:
!kimitems install -D Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_004

It is installed in a developer-friendly location, `/home/openkim/models`:

In [ ]:
!ls /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_004

The primary purpose of the KDP is to emulate the functionality of the OpenKIM pipeline by matching and running tests and verification checks. Install some tests now. The command is able to interpret incomplete names:

In [ ]:
!kimitems install -D LatticeConstantCubicEnergy_diamond_C_
!kimitems install -D ElasticConstantsCubic_diamond_C_

We can run the tests using the `pipeline-run-matches`, `pipeline-run-pair`, or `pipeline-run-verification-checks` utilities. These support wildcards:

In [ ]:
!pipeline-run-matches Tersoff*

The test output, including auxiliary files and the `results.edn` files containing the property predictions, are saved to a local directory:

In [ ]:
!ls /home/openkim/test-results

Now let's see a simple example of using the KDP to modify and test KIM models. Let's say we found a parametrization error and wanted to create a new version of the model. We will copy the old model to a new directory to begin modifying it, and remove the build files:

In [ ]:
!cp -r /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_004 /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005
!rm -r /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005/build

`CMakeLists.txt` and the metadata file `kimspec.edn` contain several references to the model name. Let's update it with the new name we chose. If we were making a more involved modification, like using a model as a template for a brand new model, the modifications would be more involved:

In [ ]:
!sed -i 's/MO_579868029681_004/MO_579868029681_005/' /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005/*

Imagine we found out that the parameter `R`, related to the cutoff, is incorrect, and needs to be increased by 0.1. Each model has a different file format. In this case, by reading the KIM documentation for the test driver:
https://openkim.org/files/MD_077075034781_005/README
we find out that the parameter file follows the LAMMPS format, and `R` is the fourth-to-last parameter:
https://docs.lammps.org/pair_tersoff.html
Let's look at the unmodified file:

In [ ]:
!cat /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005/Tersoff_1988_C.params

We see that the old value is 1.95. Let's replace it with 2.05:

In [ ]:
!sed -i 's/ 1.95 / 2.05 /' /home/openkim/models/Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005/Tersoff_1988_C.params

Let's try to re-run the elastic constants test. If the model is not built, it will automatically do so:

In [ ]:
!pipeline-run-pair ElasticConstantsCubic_diamond_C_* Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005

This causes an error! More detailed information can be found by browing the files in `/home/openkim/errors`. The issue here is that the ElasticConstants tests depends on the LatticeConstant test. Normally, the test driver queries openkim.org for this dependency, but we are using a new model that is not in the online repository. To remedy this, we need to instruct the KDP to save to and read from a local MongoDB:

In [ ]:
!pipeline-database set local

Now, we can re-run the tests in the correct order, and they will run successfully:

In [ ]:
!pipeline-run-pair LatticeConstantCubicEnergy_diamond_C_* Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005
!pipeline-run-pair ElasticConstantsCubic_diamond_C_* Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_005

When using the local database, it can be queried from the local file:

In [ ]:
from montydb import MontyClient
from bson.json_util import dumps, loads
client = MontyClient("/pipeline/db", cache_modified=0)
db = client.db
query={
    "meta.type":"tr",
    'property-id':'tag:staff@noreply.openkim.org,2014-04-15:property/structure-cubic-crystal-npt',        
}
print(loads(dumps(db.data.find(query),default=str)))